In [1]:
import os

os.chdir(r'/Users/jialunlyu/Documents/GitHub/ProjectBuffalo/')

In [2]:
import pandas as pd

In [3]:
import buffalo.ingestion as ingestion

In [4]:
ingestion.configuration.Configuration.print()

{'api_keys': {<API.ADVANTAGE: 1>: '2rrNROO0beX90lPH7ixQOp0mT_9SwF0d'}}


In [5]:
ingestor = ingestion.DataIngestion(ingestion.enum.API.ADVANTAGE)

In [6]:
tickers = ['AAPL', 'MSFT', 'IBM', 'META', 'JNJ', 'PFE', 'UNH', 'MARK', 'XLV', 'JPM', 'BAC', 'GS', 'MS', 'XLF', 'PG', 'KO', 'PEP', 'NKE', 'XLP', 'GE', 'HON', 'UTX', 'MMM', 'XLI']

#### Adjusted Daily Stock Data

In [7]:
daily_stocks_args = pd.DataFrame({
    'symbol': tickers,
    'interval': 'daily',
    'adjusted': True,
    'data_type': ingestion.enum.DataType.STOCK,
    'ingestion_type': ingestion.enum.IngestionType.REST,
    'data_name': 'ADJUSTED_DAILY_STOCK'
})

In [ ]:
ingestor.ingestion_from_web(daily_stocks_args)

In [ ]:
ingestor.store_data(r'cached_data/ingestion.sqlite', data_names='ADJUSTED_DAILY_STOCK')

#### Company Info

In [ ]:
company_info = pd.DataFrame({
    'symbol': tickers,
    'data_type': ingestion.enum.DataType.COMPANY,
    'ingestion_type': ingestion.enum.IngestionType.REST
})

In [ ]:
company_info = company_info.merge(pd.DataFrame({
    'function': ['OVERVIEW', 'INCOME_STATEMENT', 'BALANCE_SHEET', 'CASH_FLOW', 'EARNINGS', 'EARNINGS_CALENDAR'],
    'data_name': ['COMPANY_OVERVIEW', 'COMPANY_INCOME_STATEMENT', 'COMPANY_BALANCE_SHEET', 'COMPANY_CASH_FLOW', 'COMPANY_EARNINGS', 'COMPANY_EARNINGS_CALENDAR_3M']
    }), how='cross')

In [ ]:
ingestor.ingestion_from_web(company_info)

In [ ]:
ingestor.store_data(r'cached_data/ingestion.sqlite', data_names=company_info['data_name'])

#### Listing Info

In [ ]:
listing_info = pd.DataFrame({
    'date': ['2022-01-01'],
    'data_type': ingestion.enum.DataType.STOCK_LISTING,
    'ingestion_type': ingestion.enum.IngestionType.REST,
    'data_name': 'LISTING_INFO'
})

In [ ]:
ingestor.ingestion_from_web(listing_info)

In [ ]:
ingestor.store_data(r'cached_data/ingestion.sqlite', data_names=listing_info['data_name'])

#### Economic Indicators

In [ ]:
economy = pd.DataFrame({
    'function': ['FEDERAL_FUNDS_RATE', 'NONFARM_PAYROLL', 'CPI', 'UNEMPLOYMENT', 'REAL_GDP', 'REAL_GDP_PER_CAPITA'],
    'interval': ['daily', None, 'monthly', None, 'quarterly', 'quarterly'],
    'data_type': ingestion.enum.DataType.ECON,
    'ingestion_type': ingestion.enum.IngestionType.REST,
})
economy['data_name'] = economy['function']

In [ ]:
economy = pd.concat([economy, pd.DataFrame({
    'function': 'TREASURY_YIELD',
    'maturity': ['3month', '2year', '5year', '7year', '10year', '30year'],
    'interval': 'daily',
    'data_type': ingestion.enum.DataType.ECON,
    'ingestion_type': ingestion.enum.IngestionType.REST,
    'data_name': 'TREASURY_YIELD'
})], ignore_index=True)

In [ ]:
ingestor.ingestion_from_web(economy)

In [ ]:
ingestor.store_data(r'cached_data/ingestion.sqlite', data_names=economy['data_name'])

#### SMA Trend Indicator

In [ ]:
sma_trend = pd.DataFrame({
    'symbol': tickers,
    'function': 'SMA',
    'interval': 'daily',
    'data_type': ingestion.enum.DataType.TREND_INDICATOR,
    'ingestion_type': ingestion.enum.IngestionType.REST,
    'data_name': 'SMA'
})

In [ ]:
sma_trend = sma_trend.merge(pd.DataFrame({
    'time_period': [10, 50, 200]
}), how='cross').merge(pd.DataFrame({
    'series_type': ['open', 'close', 'high', 'low']
}), how='cross')

In [ ]:
ingestor.ingestion_from_web(sma_trend)

In [ ]:
ingestor.store_data(r'cached_data/ingestion.sqlite', data_names=sma_trend['data_name'])

#### Cyclic Indicators

In [ ]:
ht_sine_cycle = pd.DataFrame({
    'symbol': tickers,
    'function': 'HT_SINE',
    'interval': 'daily',
    'data_type': ingestion.enum.DataType.CYCLE_INDICATOR,
    'ingestion_type': ingestion.enum.IngestionType.REST,
    'data_name': 'HT_SINE'
})

In [ ]:
ht_sine_cycle = ht_sine_cycle.merge(pd.DataFrame({'time_period': [10, 50, 200]}), how='cross').merge(pd.DataFrame({'series_type': ['open', 'high', 'low', 'close']}), how='cross')

In [ ]:
ingestor.ingestion_from_web(ht_sine_cycle)

In [ ]:
ingestor.store_data(r'cached_data/ingestion.sqlite', data_names=ht_sine_cycle['data_name'])

#### Momentum Indicator

In [ ]:
momentum_cycle = pd.DataFrame({
    'symbol': tickers,
    'interval': 'daily',
    'data_type': ingestion.enum.DataType.CYCLE_INDICATOR,
    'ingestion_type': ingestion.enum.IngestionType.REST
})

In [ ]:
momentum_cycle = momentum_cycle.merge(pd.DataFrame({'function': ['MOM', 'ROC'], 'data_name': ['MOM', 'ROC']}), how='cross').merge(pd.DataFrame({'time_period': [10, 50, 200]}), how='cross').merge(pd.DataFrame({'series_type': ['open', 'high', 'low', 'close']}), how='cross')

In [ ]:
ingestor.ingestion_from_web(momentum_cycle)

In [ ]:
ingestor.store_data(r'cached_data/ingestion.sqlite', data_names=momentum_cycle['data_name'])

#### Market News

In [ ]:
news = pd.DataFrame({
    'symbol': tickers,
    'sort': 'Latest',
    'limit': 50,
    'data_type': ingestion.enum.DataType.MARKET_NEWS,
    'ingestion_type': ingestion.enum.IngestionType.REST,
    'data_name': 'MARKET_NEWS'
})

In [ ]:
ingestor.ingestion_from_web(news)

In [ ]:
ingestor.store_data(r'cached_data/ingestion.sqlite', data_names=news['data_name'])